In [11]:
from adaptive_mesh_refiner.adaptive_meshing import AdaptiveMeshRefiner
from src.radiation_algorithm.radiation_algorithm import radiation_algorithm
from efield.efield1 import calculate_electric_magnetic_field_at_point
from efield.efield2 import radiation_intensity_distribution_over_sphere_surface
from efield.efield3 import antenna_directivity_pattern

import numpy as np
import os
import time

In [12]:
def traitement_de_(filename):
    base_name = os.path.splitext(os.path.basename(filename))[0]
    print(f"Traitement de l'antenne {base_name}")

In [13]:
W = 0.198
H = 0.2
wc = 0.012
hc = 0.014

points = {
    'point_0' : (-W/2, -H/2),
    'point_1' : (-wc/2, -hc/2),
    'point_2' : (-wc/2, hc/2),
    'point_3' : (-W/2, H/2),
    'point_4' : (W/2, H/2),
    'point_5' : (wc/2, hc/2),
    'point_6' : (wc/2, -hc/2),
    'point_7' : (W/2, -H/2),
    'point_8' : (-wc/2, 0),
    'point_9' : (wc/2, 0)
}

# Declaration des points du contour
vertices = [points[point] for point in points]

# Indices des segments formant le contour
segments = [(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 0), (8, 9)]

# Construire le dictionnaire pour Triangle
bowtie2_antenna = dict(vertices=vertices, segments=segments)


In [14]:
feed_point_bowtie2 = [0, 0, 0]
voltage_amplitude = 1  # Valeur de l'amplitude égale à 1 volt

In [ ]:
bowtie2_name = "bowtie2"
bowtie2_folder = "data/antennas_mesh/"

# Création de l'objet de raffinage
mesh_bowtie2 = AdaptiveMeshRefiner()

# Étape 1 : Génération de la triangulation initiale avec un niveau moyen
mesh_bowtie2.initial_meshing(bowtie2_antenna, bowtie2_name, bowtie2_folder, refinement_level="high")

# show_mesh(points, triangles, feed_point_IFA)  # Change "black" si besoin
mesh_bowtie2.show_mesh(feed_point_bowtie2)

Utilisation des options de triangulation : pq20a0.01
Nombre de points = 18
Nombre de triangles = 18
Matrice points shape = (3, 18)
Matrice triangles shape = (4, 18)
Data saved successfully to data/antennas_mesh/bowtie2.mat


In [ ]:
print(points)
print(f"taille de points : {points.shape}")

In [ ]:
print(triangles)
print(f"taille de triangles : {triangles.shape}")

In [ ]:
# Normalisation pour l'aspect ratio
x_, y_, z_ = points
fig_scale = max(max(x_) - min(x_), max(y_) - min(y_))
x_scale = (max(x_) - min(x_)) / fig_scale
y_scale = (max(y_) - min(y_)) / fig_scale
z_scale = 0.3  # Z scale arbitraire pour une meilleure visualisation

# Création de la figure 3D avec Plotly
fig = ff.create_trisurf(
    x=x_,
    y=y_,
    z=z_,
    simplices=triangles[:3, :].T,
    color_func=list(range(len(triangles[:3, :].T))),  # Couleurs basées sur les indices des triangles
    show_colorbar=True,
    title="Maillage triangulaire",
    aspectratio=dict(x=x_scale, y=y_scale, z=z_scale)
)
# Afficher la figure
fig.show()

In [ ]:
# Crée un dictionnaire contenant toutes les données à sauvegarder
data = {
    'p' : points,
    't' : triangles
}
save_folder_name = 'data/antennas_mesh/'
save_file_name = 'adaptative_bowtie.mat'
full_save_path = os.path.join(save_folder_name, save_file_name)  # Chemin complet pour la sauvegarde

# Vérifie si le dossier existe, sinon crée le dossier
if not os.path.exists(save_folder_name): # Vérification et création du dossier si nécessaire
    os.makedirs(save_folder_name)
    print(f"Directory '{save_folder_name}' created.")

# Sauvegarde les données dans le fichier MAT
savemat(full_save_path, data)
print(f"Data saved successfully to {full_save_path}")

In [ ]:
filename_adaptative_mesh_bowtie = 'data/antennas_mesh/adaptative_bowtie.mat'

In [ ]:
wave_incident_direction = np.array([0, 0, -1])

In [ ]:
start_time = time.time()

traitement_de_(filename_adaptative_mesh_bowtie)
scattering_algorithm(filename_adaptative_mesh_bowtie, 75e7, wave_incident_direction, polarization=np.array([0, 1, 0]), load_from_matlab=False)

elapsed_time = time.time() - start_time
print(f"Temps écoulé pour le traitement de l'antenne étudiée : {elapsed_time:.6f} secondes")
print('\n')

In [ ]:
filename_adaptative_mesh_bowtie_current = 'data/antennas_current/adaptative_bowtie_current.mat'
filename_adaptative_mesh_bowtie_mesh1 = 'data/antennas_mesh1/adaptative_bowtie_mesh1.mat'

In [ ]:
*_, current = DataManager_rwg4.load_data(filename_adaptative_mesh_bowtie_current)
*_, edges = DataManager_rwg1.load_data(filename_adaptative_mesh_bowtie_mesh1)
print(current.shape)
print(edges.first_points.shape)
print(edges.second_points.shape)


In [ ]:
current_norm = np.abs(current)
print(current_norm.shape)
average_current_norm = np.mean(current_norm)
print(f"Le courant moyen sur la surface de l'antenne est : {average_current_norm} A")

In [ ]:
# Fonction pour estimer l'erreur basée sur les courants de surface
def estimate_error(triangles, current_norm, average_current_norm):
    errors = np.zeros(len(triangles.T))
    for i in range(len(triangles.T)):
        # Calculer l'erreur pour les courants supérieurs au courant moyen
        current_value = current_norm[i]
        if current_value > average_current_norm:
            errors[i] = current_value - average_current_norm
    return errors

In [ ]:
def estimate_error(triangles, edges, current_norm, average_current_norm):
    """
    Estime les erreurs au niveau des triangles en se basant sur les courants des arêtes.

    Paramètres :
        - triangles (ndarray) : Les indices des sommets définissant les triangles (shape : 3, n_triangles).
        - edges (Edges) : L'objet Edges contenant les arêtes et leurs relations avec les points.
        - current_norm (ndarray) : Les normes des courants calculées pour chaque arête (shape : n_edges).
        - average_current_norm (float) : La norme moyenne des courants.

    Retourne :
        - errors (ndarray) : Un tableau contenant les erreurs estimées pour chaque triangle.
    """
    errors = np.zeros(triangles.shape[1])  # Une erreur par triangle
    edge_currents = {}

    # Associe les courants aux arêtes
    for i in range(edges.total_number_of_edges):
        edge_currents[(edges.first_points[i], edges.second_points[i])] = current_norm[i]
        edge_currents[(edges.second_points[i], edges.first_points[i])] = current_norm[i]  # Pour gérer les deux orientations

    # Calcule l'erreur pour chaque triangle
    for i, triangle in enumerate(triangles.T):
        triangle_edges = [
            (triangle[0], triangle[1]),
            (triangle[1], triangle[2]),
            (triangle[2], triangle[0])
        ]
        # Moyenne des courants des arêtes du triangle
        triangle_current = np.mean([edge_currents[edge] for edge in triangle_edges])
        if triangle_current > average_current_norm:
            errors[i] = triangle_current - average_current_norm

    return errors


In [ ]:
errors = estimate_error(triangles, edges, current_norm, average_current_norm)
print(errors.shape)
print(errors)

In [ ]:
threshold = 1e-03

In [ ]:
def refine_mesh(points, triangles, errors, threshold):
    refine_indices = np.where(errors > threshold)[0]
    for idx in refine_indices:
        triangle = triangles[:3, idx]    
        midpoints = (points[:, triangle] + points[:, np.roll(triangle, -1)]) / 2
        points = np.hstack([points, midpoints])
    return points[:2, :].T

In [ ]:
# mesh_adaptative_bowtie['vertices'] = refine_mesh(triangles, errors, threshold)
mesh_adaptative_bowtie['vertices'] = refine_mesh(points, triangles, errors, threshold)
mesh_adaptative_bowtie_2 = tr.triangulate(mesh_adaptative_bowtie, 'pq30a0.00001Dj')

In [ ]:
# Extraire les sommets et les triangles
points = np.zeros((3, mesh_adaptative_bowtie_2['vertices'].shape[0]))
points[:2, :] = np.array(mesh_adaptative_bowtie_2['vertices']).T
triangles = np.ones((4, mesh_adaptative_bowtie_2['triangles'].shape[0]), dtype=int)
triangles[:3, :] = np.array(mesh_adaptative_bowtie_2['triangles']).T


# Normalisation pour l'aspect ratio
x_, y_, z_ = points
fig_scale = max(max(x_) - min(x_), max(y_) - min(y_))
x_scale = (max(x_) - min(x_)) / fig_scale
y_scale = (max(y_) - min(y_)) / fig_scale
z_scale = 0.3  # Z scale arbitraire pour une meilleure visualisation

# Création de la figure 3D avec Plotly
fig = ff.create_trisurf(
    x=x_,
    y=y_,
    z=z_,
    simplices=triangles[:3, :].T,
    color_func=list(range(len(triangles[:3, :].T))),  # Couleurs basées sur les indices des triangles
    show_colorbar=True,
    title="Maillage triangulaire",
    aspectratio=dict(x=x_scale, y=y_scale, z=z_scale)
)
# Afficher la figure
fig.show()